### 基礎設定

In [ ]:
# 使用 LangChain 1.0 串接本地 大模型
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="qwen3:latest",
    temperature=0.5,
)

result = llm.invoke("What is the capital of the moon?")
print(result.content)

### Human In The Loop: LangGraph Version Recall

In [ ]:
from typing import TypedDict
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import InMemorySaver

# 定義 State
class State(TypedDict):
    content: str        # LLM 生成的內容
    decision: str       # 人類的決策 ('approve', 'reject', 'continue'...)
    feedback: str       # 拒絕時的理由 (可選)

# 節點：LLM 生成 (這裡簡化模擬)
def generator_node(state: State):
    print("--- 🤖 LLM 正在生成內容 ---")
    # 實際這裡會寫 llm.invoke(...)
    # TODO: 增加你有興趣的內容
    return {"content": "這是 LLM 寫好的初稿，請審核。"}

# 節點：人類審核 (這是一個中斷點，什麼都不用做)
def human_review_node(state: State):
    pass

# 邊：邏輯判斷：根據人類的 decision 決定下一步
def router(state: State):
    decision = state.get("decision")
    
    if decision == "reject":
        return "rejected"  # 走向拒絕的結尾
    else:
        return "approved"  # 包含 approve 和 edit (視為通過)

# 節點：後續處理 (模擬)
def approved_node(state: State):
    print(f"--- ✅ 內容已發布: {state['content']} ---")
    return

def rejected_node(state: State):
    print(f"--- ❌ 內容被拒絕，流程終止。理由: {state.get('feedback')} ---")
    return

# --- 建構 Graph ---
builder = StateGraph(State)

builder.add_node("generator", generator_node)
builder.add_node("human_review", human_review_node)
builder.add_node("approved", approved_node)
builder.add_node("rejected", rejected_node)

builder.set_entry_point("generator")
builder.add_edge("generator", "human_review")

builder.add_conditional_edges(
    "human_review",
    router,
    {
        "approved": "approved",
        "rejected": "rejected"
    }
)

# HITL 邏輯流程:
# (LLM 生成)
#     |
# [Human Review] <--- 🛑 Interrupt Here
#     |
#     +--- 操作: Edit/Approve (update_state)
#     |       |
#     |       v
#     |   [Router] --(Pass)--> [✅ Approved Node]
#     |
#     +--- 操作: Reject (update_state)
#             |
#             v
#         [Router] --(Fail)--> [❌ Rejected Node]

graph = builder.compile(checkpointer=InMemorySaver(), interrupt_before=["human_review"])

In [ ]:
import uuid

# 設定一個 Thread ID
config = {"configurable": {"thread_id": str(uuid.uuid4())}}

# 初始輸入
input_to_stream = {"content": "", "decision": None}

print("🚀 流程啟動！請關注下方的輸出與輸入框...\n")

while True:
    # 讓 Graph 跑起來 (直到遇到中斷或結束)
    for event in graph.stream(input_to_stream, config):
        pass 
    
    # 檢查目前狀態
    snapshot = graph.get_state(config)

    # print(f"DEBUG: Next step is {snapshot.next}")
    
    # 如果 next 為空，代表流程結束
    if not snapshot.next:
        print("\n🎉 流程已完整結束。")
        break
    
    # 檢測是否停在我們預期的節點
    if snapshot.next[0] == "human_review":
        current_content = snapshot.values['content']
        
        print(f"\n🛑 [中斷] 等待人類審核。")
        print(f"📄 當前內容: {current_content}")
        print("-" * 30)
        
        
        print("請選擇操作：")
        print("1. ✅ Approve (直接通過)")
        print("2. ✏️ Edit (修改內容後通過)")
        print("3. ❌ Reject (拒絕並填寫理由)")
        
        user_choice = input("請輸入選項 (1/2/3): ")
        
        # 根據你的輸入，執行不同的 update_state
        if user_choice == "1":
            print("--> 你選擇了 Approve。")
            graph.update_state(config, {"decision": "approve"})
            
        elif user_choice == "2":
            new_content = input("請輸入修改後的內容: ")
            print(f"--> 你選擇了 Edit，內容更新為: {new_content}")
            graph.update_state(config, {
                "content": new_content, 
                "decision": "approve"
            })
            
        elif user_choice == "3":
            reason = input("請輸入拒絕理由: ")
            print(f"--> 你選擇了 Reject，理由: {reason}")
            graph.update_state(config, {
                "decision": "reject",
                "feedback": reason
            })
            
        else:
            print("⚠️ 無效輸入，預設視為 Approve。")
            graph.update_state(config, {"decision": "approve"})
            
        # 關鍵：將 input 設為 None，代表 Resume (繼續執行)
        input_to_stream = None

### HITL Langchain 1.0 Integration

In [ ]:
# 定義工具
from langchain.tools import tool
import requests
import json
from datetime import datetime
import os

@tool
def get_weather(lat, lon):
    """
    查詢即時天氣的函式。

    :param lat: 必要參數，浮點數 (float) 類型，
                用來表示要查詢天氣的具體緯度。

    :param lon: 必要參數，浮點數 (float) 類型，
                用來表示要查詢天氣的具體經度。

    :return: 來自 Open Metro API 的即時天氣查詢結果。
            (請求 URL: https://api.open-meteo.com/v1/forecast)

            回傳的物件是「解析後的 JSON 格式物件」，並以「字串」形式表示，
            其中包含了所有重要的天氣資訊。
    :rtype: str
    """

    url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&hourly=temperature_2m"

    response = requests.get(url)
    
    data = response.json()
    temp = data['hourly']['temperature_2m'][0]
    return {"temperature": temp}

@tool
def write_file(content: str, filename: str = "") -> str:
    """
    將指定內容寫入本地文件。

    :param content: 必要參數，字串 (str) 類型，
                    用來表示需要寫入文件的具體內容。
    :param filename: 可選參數，字串 (str) 類型，
                    建議提供空字串，則會自動生成一個文件名。

    :return: 寫入結果提示資訊。
    """
    try:
        if not filename:
            # ✅ 始終先定義文件名（防止未綁定變量）
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"output_{timestamp}.txt"

        # 寫入文件
        with open(filename, "w", encoding="utf-8") as f:
            f.write(content)

        abs_path = os.path.abspath(filename)
        return f"✅ 已成功寫入本地文件：{abs_path}"

    except Exception as e:
        return f"❌ 文件寫入失敗：{str(e)}"

In [ ]:
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware 
from langgraph.checkpoint.memory import InMemorySaver 
from langchain_ollama import ChatOllama


agent = create_agent(
    model=ChatOllama(
        model="qwen3:latest",
        temperature=0.5,
    ),
    tools=[write_file, get_weather],
    middleware=[
        HumanInTheLoopMiddleware( 
            interrupt_on={
                # "write_file": True,  # All decisions (approve, edit, reject) allowed
                "get_weather": {"allowed_decisions": ["approve", "reject"]},  # No editing allowed
            },
            # Prefix for interrupt messages - combined with tool name and args to form the full message
            # e.g., "Tool execution pending approval: execute_sql with query='DELETE FROM...'"
            # Individual tools can override this by specifying a "description" in their interrupt config
            description_prefix="Tool execution pending approval",
        ),
    ],
    # Human-in-the-loop requires checkpointing to handle interrupts.
    # In production, use a persistent checkpointer like AsyncPostgresSaver.
    checkpointer=InMemorySaver(),  
)

In [ ]:
from langgraph.types import Command
import uuid

# Human-in-the-loop leverages LangGraph's persistence layer.
# You must provide a thread ID to associate the execution with a conversation thread,
# so the conversation can be paused and resumed (as is needed for human review).
config = {"configurable": {"thread_id": str(uuid.uuid4())}} 
# Run the graph until the interrupt is hit.
result = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "what is the weather in taipei?"
            }
        ]
    },
    config=config 
)

In [ ]:
# The interrupt contains the full HITL request with action_requests and review_configs
print(result['__interrupt__'])

In [ ]:
# Review the process
result

In [ ]:
# Resume with approval decision
result = agent.invoke(
    Command( 
        resume={"decisions": [
                    {"type": "approve"}, 
                ]}  # or "edit", "reject"
    ), 
    config=config # Same thread ID to resume the paused conversation
)

In [ ]:
result

### 練習

* HITL 修改參數內容
* HITL 互動模式